# First Kaggle competition - Image classification

To complete this challenge I opted to start with a CNN inspired on VGG-11 simplifying the FC layers at the top thus reducing the number of parameters (the majority resides in the classification layers) and so the tendency to overfit.

The results are encouraging but I don't think that is possible to obtain much more from a network trained from scratch due to the small dataset.
So my next try will use transfer learning and fine tuning.

I used a VGG-16 already trained with imagenet dataset and I appended at the end another convolutional layer plus a dense net to perform classification.
With TL and some hyperparameter tuning the model was able to reach a score of 0.89333 on the test set. I think the model can be improved by analyzing the activations in the last layer of the network to understand which other layers to include, remove or modify.

My last try was with a unfreezed Xception network, the classification layer that I added at the end is similar to the one used in previous try but now I've let all the network's layers to be trained.
Obviously I had to increase the learning rate so I opted to use also the ReduceLROnPlateau callback to better approach the local minimum.
This model was able to achieve a score of 0.92222.

Unfortunately I was not able to find a teammate because I started some days after the publication of the challenge.

During this challenge I used, instead of TensorBoard, Weight and Biases to have statistics about how the model is performing and to perform hyperparameters tuning in a simpler way. Here you can find the entire project with all the runs and sweeps: https://wandb.ai/lrsb/kaggle1


# Download dataset

In [ ]:
import json

!pip install --upgrade --force-reinstall --no-deps kaggle
!pip install --upgrade wandb

#@markdown Insert here your credentials
kaggle_username = ''#@param {type:'string'}
kaggle_api_key = ''#@param {type:'string'}
wandb_key = ''#@param {type:'string'}

!wandb login {wandb_key}

api_token = {'username': kaggle_username, 'key': kaggle_api_key}

!mkdir ~/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as kaggle_json:
  json.dump(api_token, kaggle_json)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c artificial-neural-networks-and-deep-learning-2020
!unzip -q artificial-neural-networks-and-deep-learning-2020.zip

!mkdir ./MaskDataset/train ./MaskDataset/train/0 ./MaskDataset/train/1 ./MaskDataset/train/2

# Setup

### Splitting dataset

In [ ]:
import json, shutil, os

cwd = os.getcwd()

dataset_dir = os.path.join(cwd, 'MaskDataset')
training_dir = os.path.join(dataset_dir, 'training')
split_dir = os.path.join(dataset_dir, 'train')

with open(os.path.join(dataset_dir, 'train_gt.json')) as js:
  data = json.load(js)
  files = os.listdir(training_dir)

  for f in files:
    start_dir = os.path.join(training_dir, f)
    end_dir = os.path.join(os.path.join(split_dir, str(data[f])), f)
    shutil.move(start_dir, end_dir)

### Making results more reproducible and setting params

In [ ]:
import tensorflow as tf

SEED = 1234#@param {type:'number'}
tf.random.set_seed(SEED)

num_classes = 3
classes = ['none', 'all', 'some']

img_w = 299#@param {type:'number'}
img_h = 299#@param {type:'number'}

input_shape = (img_h, img_w, 3)

### Code for creating datasets

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def GetDatasets(data_augmentation, validation_split, batch_size, preprocessing_function):
  if data_augmentation:
    image_generator = ImageDataGenerator(rotation_range=30,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        rescale=1./255,
                                        validation_split=validation_split,
                                        preprocessing_function=preprocessing_function)
  else:
    image_generator = ImageDataGenerator(rescale=1./255,
                                        validation_split=validation_split,
                                        preprocessing_function=preprocessing_function)
    
  test_img_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessing_function)
    
  train_gen = image_generator.flow_from_directory(split_dir,
                                                  batch_size=batch_size,
                                                  target_size=(img_h, img_w),
                                                  color_mode='rgb',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  subset='training',
                                                  seed=SEED)

  valid_gen = image_generator.flow_from_directory(split_dir,
                                                  batch_size=batch_size,
                                                  target_size=(img_h, img_w),
                                                  color_mode='rgb',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  subset='validation',
                                                  seed=SEED)

  test_gen = test_img_generator.flow_from_directory(dataset_dir,
                                                    batch_size=1,
                                                    target_size=(img_h, img_w),
                                                    color_mode='rgb',
                                                    classes=['test'],
                                                    shuffle=False,
                                                    seed=SEED)

  train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=([None, img_w, img_h, 3], [None, num_classes]))
  train_dataset = train_dataset.repeat()


  valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=([None, img_w, img_h, 3], [None, num_classes]))
  valid_dataset = valid_dataset.repeat()


  test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=([None, img_w, img_h, 3], [None, num_classes]))
  test_dataset = test_dataset.repeat()

  return train_dataset, train_gen, valid_dataset, valid_gen, test_dataset, test_gen

### Code for saving testset results

In [ ]:
import ntpath

def PredictDataset(model, dataset, directory):
  predictions = model.predict_generator(dataset, len(dataset), verbose=1)
  filenames = dataset.filenames

  results = {}
  i = 0
  for p in predictions:
    results[ntpath.basename(filenames[i])] = str(np.argmax(p))
    i = i + 1

  with open(os.path.join(directory, 'results.csv'), 'w') as f:
    f.write('Id,Category\n')
    for key, value in results.items():
      f.write(key + ',' + str(value) + '\n')

# (1) CNN based on VGG-11 architecture
### Kaggle score: 0.81555



In [ ]:
import wandb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.tensorflow.keras.optimizers import RMSprop
from wandb.keras import WandbCallback
from tensorflow.keras.applications.vgg16 import preprocess_input

def GetVGG11BasedCNN(learning_rate):
  model = Sequential()

  # CNN-1: conv3-64 + maxpool
  model.add(Convolution2D(64, 3, padding='same', input_shape=input_shape, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # CNN-2: conv3-128 + maxpool
  model.add(Convolution2D(128, 3, padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # CNN-3: conv3-256x2 + maxpool
  model.add(Convolution2D(256, 3, padding='same', activation='relu'))
  model.add(Convolution2D(256, 3, padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # CNN-4: conv3-512x2 + maxpool
  model.add(Convolution2D(512, 3, padding='same', activation='relu'))
  model.add(Convolution2D(512, 3, padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # CNN-5: conv3-512x2 + maxpool
  model.add(Convolution2D(512, 3, padding='same', activation='relu'))
  model.add(Convolution2D(512, 3, padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # FC-1024x2 Fully connected layers
  model.add(Flatten())
  model.add(Dense(units=1024, activation='relu'))
  model.add(Dense(units=1024, activation='relu'))

  # FC-3 Last layer
  model.add(Dense(units=num_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', 
                optimizer=RMSprop(learning_rate=learning_rate),
                metrics=['accuracy'])
  return model

#@markdown Set hyperparameters used during training

data_augmentation = True#@param {type:'boolean'}
validation_split = 0.1#@param {type:'number'}
batch_size = 8#@param {type:'number'}
epochs = 100#@param {type:'number'}
use_early_stopping = True#@param {type:'boolean'}
learning_rate = 1e-5#@param {type:'number'}

#@markdown Or use hyperparameter optimization (done on WandB platform)

use_hyperparameter_optimization = False#@param {type:'boolean'}

!nvidia-smi

if use_hyperparameter_optimization:
  def RunFitWithHypOpt():
    wandb.init()
    datasets = GetDatasets(wandb.config.data_augmentation,
                           wandb.config.validation_split,
                           wandb.config.batch_size,
                           preprocess_input)
    model = GetVGG11BasedCNN(wandb.config.learning_rate)
    
    model.fit(x=datasets[0],
              epochs=wandb.config.epochs,
              steps_per_epoch=len(datasets[1]),
              validation_data=datasets[2],
              validation_steps=len(datasets[3]), 
              callbacks=[EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True),
                         WandbCallback(data_type='image', labels=classes)])

    PredictDataset(model, datasets[5], wandb.run.dir)

  # Initialise WandB agent that will perform HP optimization
  wandb.agent(wandb.sweep({
    'description': 'VGG11 based CNN optimization',
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 3
    },
    'parameters': {
          'data_augmentation': {
              'values': [True, False]
          },
          'validation_split': {
              'values': [0.1, 0.2]
          },
          'batch_size': {
              'values': [4, 8, 16, 32, 64]
          },
          'epochs': {
              'value': 100
          },
          'learning_rate': {
              'values': [1e-5, 5e-5, 1e-4]
          }
      }
  }, project='kaggle1'), function=RunFitWithHypOpt)

else:
  wandb.init(project='kaggle1', config={
      'data_augmentation': data_augmentation,
      'validation_split': validation_split,
      'batch_size': batch_size,
      'epochs': epochs,
      'use_early_stopping': use_early_stopping,
      'learning_rate': learning_rate
  })

  callbacks = [WandbCallback(data_type='image', labels=classes)]
  if use_early_stopping:
    callbacks.append(EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True))

  datasets = GetDatasets(data_augmentation,
                         validation_split,
                         batch_size,
                         preprocess_input)
  model = GetVGG11BasedCNN(learning_rate)
  model.summary()
  
  model.fit(x=datasets[0],
            epochs=epochs,
            steps_per_epoch=len(datasets[1]),
            validation_data=datasets[2],
            validation_steps=len(datasets[3]), 
            callbacks=callbacks)

  PredictDataset(model, datasets[5], wandb.run.dir)

### Results

Download model [here](https://drive.google.com/file/d/1-50J64ZxbJETPqTCPl8xH7ZALow-uSyo/view?usp=sharing)

Details of the run [here](https://wandb.ai/lrsb/kaggle1/runs/3k466bg5)

# (2) Transfer learning and fine tuning on VGG16 
### Kaggle score: 0.89333


In [ ]:
import wandb
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from wandb.keras import WandbCallback
from tensorflow.keras.applications.vgg16 import preprocess_input

def GetModel(learning_rate, filters, units_1, units_2, ft_level):
  vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

  for layer in vgg16.layers[:ft_level]:
    layer.trainable =  False

  model = Sequential()
  model.add(vgg16)

  model.add(Conv2D(filters, 3, padding='valid', activation='relu'))
  model.add(Dense(units=units_1, activation='relu'))
  model.add(GlobalAveragePooling2D())

  model.add(Dense(units=units_2, activation='relu'))
  model.add(Dense(units=num_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', 
                optimizer=Adam(learning_rate),
                metrics=['accuracy'])
  return model

#@markdown Set hyperparameters used during training

data_augmentation = True#@param {type:'boolean'}
validation_split = 0.1#@param {type:'number'}
batch_size = 8#@param {type:'number'}
epochs = 100#@param {type:'number'}
use_early_stopping = True#@param {type:'boolean'}
learning_rate = 1e-5#@param {type:'number'}
filters = 16#@param {type:'number'}
units_1 = 128#@param {type:'number'}
units_2 = 256#@param {type:'number'}
ft_level = 13#@param {type:'number'}

#@markdown Or use hyperparameter optimization (done on WandB platform)

use_hyperparameter_optimization = False#@param {type:'boolean'}

!nvidia-smi

if use_hyperparameter_optimization:
  def RunFitWithHypOpt():
    wandb.init()
    datasets = GetDatasets(wandb.config.data_augmentation,
                           wandb.config.validation_split,
                           wandb.config.batch_size,
                           preprocess_input)
    model = GetModel(wandb.config.learning_rate,
                     wandb.config.filters,
                     wandb.config.units_1,
                     wandb.config.units_2,
                     wandb.config.ft_level)
    
    model.fit(x=datasets[0],
              epochs=wandb.config.epochs,
              steps_per_epoch=len(datasets[1]),
              validation_data=datasets[2],
              validation_steps=len(datasets[3]), 
              callbacks=[EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True),
                         WandbCallback(data_type='image', labels=classes)])

    PredictDataset(datasets[5], wandb.run.dir)

  # Initialise WandB agent that will perform HP optimization
  wandb.agent(wandb.sweep({
    'description': 'VGG16 TL optimization',
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 3
    },
    'parameters': {
          'data_augmentation': {
              'values': [True, False]
          },
          'validation_split': {
              'values': [0.1, 0.2]
          },
          'batch_size': {
              'values': [4, 8, 16, 32, 64]
          },
          'epochs': {
              'value': 50
          },
          'learning_rate': {
              'values': [1e-5, 5e-5, 1e-4]
          },
          'filter': {
              'values': [16, 32, 64]
          },
          'units_1': {
              'values': [128, 256, 512]
          },
          'units_2': {
              'values': [64, 126, 256]
          },
          'ft_level': {
              'values': [13, 14]
          }
      }
  }, project='kaggle1'), function=RunFitWithHypOpt)

else:
  wandb.init(project='kaggle1', config={
      'data_augmentation': data_augmentation,
      'validation_split': validation_split,
      'batch_size': batch_size,
      'epochs': epochs,
      'use_early_stopping': use_early_stopping,
      'learning_rate': learning_rate,
      'filter': filter,
      'units_1': units_1,
      'units_2': units_2,
      'ft_level': ft_level
  })

  callbacks = [WandbCallback(data_type='image', labels=classes)]
  if use_early_stopping:
    callbacks.append(EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True))

  datasets = GetDatasets(data_augmentation,
                         validation_split,
                         batch_size,
                         preprocess_input)
  model = GetModel(learning_rate,
                   filters,
                   units_1,
                   units_2,
                   ft_level)
  
  model.fit(x=datasets[0],
            epochs=epochs,
            steps_per_epoch=len(datasets[1]),
            validation_data=datasets[2],
            validation_steps=len(datasets[3]), 
            callbacks=callbacks)

  PredictDataset(datasets[5], wandb.run.dir)

### Results

Download model [here](https://drive.google.com/file/d/1gSlj2il_R2kc0F_3aWZildpUS9gKgToJ/view?usp=sharing)

Details of the run [here](https://wandb.ai/lrsb/kaggle1/runs/o6vogtpx)

# (3) Transfer learning and retuning of Xception with ReduceLROnPlateau
### Kaggle score: 0.92222

In [ ]:
import wandb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
from tensorflow.keras.applications.xception import preprocess_input

def GetModel(learning_rate):
  xcp = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

  model = Sequential()
  model.add(xcp)
  model.add(Conv2D(32, 3, padding='valid', activation='relu'))
  model.add(Conv2D(32, 3, padding='valid', activation='relu'))
  model.add(Dense(units=256, activation='relu'))
  model.add(GlobalAveragePooling2D())

  model.add(Dense(units=128, activation='relu'))
  model.add(Dense(units=num_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate),
                metrics=['accuracy'])
  return model

#@markdown #MODIFY DATASET SHAPE TO 299x299 TO FIT XCEPTION INPUT SHAPE (SETUP SECTION)

#@markdown Set hyperparameters used during training

data_augmentation = True#@param {type:'boolean'}
validation_split = 0.1#@param {type:'number'}
batch_size = 32#@param {type:'number'}
epochs = 50#@param {type:'number'}
use_early_stopping = True#@param {type:'boolean'}
learning_rate = 1e-3#@param {type:'number'}

!nvidia-smi

wandb.init(project='kaggle1', config={
    'data_augmentation': data_augmentation,
    'validation_split': validation_split,
    'batch_size': batch_size,
    'epochs': epochs,
    'use_early_stopping': use_early_stopping,
    'learning_rate': learning_rate
})

callbacks = [WandbCallback(data_type='image', labels=classes)]

if use_early_stopping:
  callbacks.append(EarlyStopping(monitor='val_accuracy',
                                 mode='max',
                                 verbose=1,
                                 patience=8,
                                 restore_best_weights=True))
  
callbacks.append(ReduceLROnPlateau(monitor='val_accuracy',
                                   mode='max',
                                   factor=0.1,
                                   patience=3,
                                   cooldown=1,
                                   min_delta=0,
                                   verbose=1))

datasets = GetDatasets(data_augmentation,
                        validation_split,
                        batch_size,
                        preprocess_input)
model = GetModel(learning_rate)

model.fit(x=datasets[0],
          epochs=epochs,
          steps_per_epoch=len(datasets[1]),
          validation_data=datasets[2],
          validation_steps=len(datasets[3]), 
          callbacks=callbacks)

PredictDataset(model, datasets[5], wandb.run.dir)

### Results

Download model [here](https://drive.google.com/file/d/1gSlj2il_R2kc0F_3aWZildpUS9gKgToJ/view?usp=sharing)

Details of the run [here](https://wandb.ai/lrsb/kaggle1/runs/age3svuk)

# Utilities

### Predict dataset using a saved model

In [ ]:
from tensorflow import keras

model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Kaggle1/vgg16-model-best.h5')

datasets = GetDatasets(True, 0.1, 8, keras.applications.vgg16.preprocess_input)

PredictDataset(model, datasets[5], '/content')